In [1]:
import warnings
warnings.filterwarnings('ignore') 

warnings.warn("This warning will be hidden")
print("Script continues...")

Script continues...


In [2]:

import torch
from transformers import GPT2Config, GPT2LMHeadModel, Trainer, TrainingArguments
from transformers.data.data_collator import DataCollatorForLanguageModeling
import sentencepiece as spm
from datasets import load_dataset
import math


print("data loading...")
dataset = load_dataset("text", data_files={"train": "/home/dixit/Project/May try/hindi_corpus_fixed.txt"})
print("data loaded!!!")

2025-05-16 16:07:03.867477: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747391824.976364    4239 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747391825.381399    4239 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747391828.006792    4239 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747391828.006866    4239 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1747391828.006879    4239 computation_placer.cc:177] computation placer alr

data loading...
data loaded!!!


In [3]:
from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("/home/dixit/Project/May try/my_hindi_t5_tokenizer")
tokenizer.model_max_length = 256
tokenizer.pad_token = "<pad>"  
tokenizer.padding_size = "right"



def tokenize_function(examples):
    # return {"input_ids": tokenizer.EncodeAsIds(examples["text"])}
    tokens = tokenizer(examples["text"], truncation=True)
    return {"input_ids": tokens["input_ids"]}

print("tokenizing...")
tokenized_datasets = dataset.map(
    tokenize_function,
    batched=True,
    remove_columns=["text"],
    batch_size=500,
    num_proc=8,  # Adjust based on CPU cores
    load_from_cache_file=True
)
print("tokenizing completed!!!")


tokenizing...
tokenizing completed!!!


In [4]:
# 3. Model Configuration (Optimized for 6GB GPU)
config = GPT2Config(
    vocab_size = tokenizer.vocab_size,
    n_positions=256,
    n_ctx=256,
    n_embd=512,
    n_layer=6,
    n_head=8,
    bos_token_id = tokenizer.bos_token_id,
    eos_token_id = tokenizer.eos_token_id,
    layer_norm_epsilon=1e-5,
    initializer_range=0.02,
    summary_activation='tanh',
    use_cache=False,
    dropout= 0.1,
)


In [5]:
model = GPT2LMHeadModel(config)
model.config.pad_token_id = tokenizer.pad_token_id  
model.gradient_checkpointing_enable()  

In [6]:
# 4. Training Arguments with Checkpointing

training_args = TrainingArguments(
    output_dir="/home/dixit/Project/May try/hindi-lm-new",
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=6,  
    per_device_eval_batch_size=2,  
    gradient_accumulation_steps=4,  
    learning_rate=2e-4,
    optim="adafactor",
    max_grad_norm=1.0,
    save_total_limit=100000,
    logging_dir="/home/dixit/Project/May try/logs-new",
    eval_strategy ="steps", 
    save_strategy="epoch",
    logging_steps=1000,
    eval_steps=5000, 
    warmup_steps=1000,
    save_steps = 10000,
    weight_decay=0.01,
    dataloader_num_workers=2,
    remove_unused_columns=False, 
    resume_from_checkpoint=True,  
    fp16=True,  
    eval_accumulation_steps=None,
    report_to="tensorboard",
)


In [7]:
# Peek at a batch
batch = tokenized_datasets["train"][0]
print("Input IDs max:", max(batch["input_ids"]))
print("Label max (if applicable):", max(batch["labels"]) if "labels" in batch else "N/A")
print("Tokenizer vocab size:", tokenizer.vocab_size)
print("Model vocab size:", model.get_input_embeddings().weight.shape[0])

Input IDs max: 28052
Label max (if applicable): N/A
Tokenizer vocab size: 30000
Model vocab size: 30000


In [8]:
# model.resize_token_embeddings(tokenizer.vocab_size)

In [ ]:
# 5. Custom Metrics
# import math

# def compute_metrics(eval_preds):
#     logits, labels = eval_preds
#     shift_logits = logits[..., :-1, :].contiguous()
#     shift_labels = labels[..., 1:].contiguous()

#     loss_fct = torch.nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)
#     loss = loss_fct(shift_logits.view(-1, shift_logits.size(-1)), shift_labels.view(-1))
#     return {"perplexity": math.exp(loss.item())}


# 6. Create Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    pad_to_multiple_of=8
)


import numpy as np
seed = 42
np.random.seed(seed)
torch.manual_seed(seed)
train_dataset_1 = tokenized_datasets["train"]

# Shuffle and select a fixed subset for eval_dataset
eval_dataset_1 = train_dataset_1.shuffle(seed=seed).select(range(2000))

# 7. Start Training
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset_1,
    eval_dataset=eval_dataset_1,
    compute_metrics=None,
)


def print_gpu_usage():
    print(f"GPU Memory Allocated: {torch.cuda.memory_allocated()/1024**2:.2f} MB")
    print(f"GPU Memory Cached: {torch.cuda.memory_reserved()/1024**2:.2f} MB")

print_gpu_usage()


trainer.train(resume_from_checkpoint=False)


GPU Memory Allocated: 132.63 MB
GPU Memory Cached: 148.00 MB


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss,Validation Loss


In [ ]:
print("Tokenizer vocab size:", tokenizer.vocab_size)

In [ ]:
print("Model vocab size:", model.get_input_embeddings().weight.shape[0])

In [ ]:
# # Part 2: Text Generation Evaluation (generate.py)
# from transformers import pipeline

# def generate_text(prompt, max_length=50):
#     generator = pipeline("text-generation", model="/content/drive/MyDrive/hindi-lm", tokenizer=tokenizer)
#     result = generator(
#         prompt,
#         max_length=max_length,
#         num_return_sequences=1,
#         pad_token_id=tokenizer.pad_id() or tokenizer.eos_id(),
#     )
#     return result[0]["generated_text"]

# # Example usage
# print(generate_text("भारत एक महान देश है"))

In [ ]:
# Part 3: Text Classification Evaluation (classifier.py)
# from transformers import GPT2ForSequenceClassification
# from sklearn.metrics import accuracy_score, f1_score

# 1. Load Fine-tuned Model
# clf_model = GPT2ForSequenceClassification.from_pretrained("/content/drive/MyDrive/hindi-lm", num_labels=num_classes)

# 2. Evaluation Function
# def evaluate_classification(test_dataset):
#     trainer = Trainer(
#         model=clf_model,
#         args=TrainingArguments(
#             output_dir="./tmp",
#             per_device_eval_batch_size=8
#         ),
#         compute_metrics=lambda p: {
#             "accuracy": accuracy_score(p.label_ids, p.predictions.argmax(-1)),
#             "f1": f1_score(p.label_ids, p.predictions.argmax(-1), average="weighted")
#         }
#     )
#     return trainer.evaluate(test_dataset)

# Part 4: Indic Eval Integration (optional)
# from indic_eval import evaluate

# # For classification tasks
# def run_indic_eval(dataset):
#     return evaluate(
#         task="text-classification",
#         model=clf_model,
#         tokenizer=tokenizer,
#         dataset=dataset,
#         metrics=["accuracy", "f1"]
#     )

# Hardware Notes:
# - Batch size can be increased if memory allows
# - Use gradient_accumulation_steps for better batch utilization
# - FP16 helps reduce memory usage
# - Consider using DeepSpeed if needed

# Resume Training:
# Just rerun the script - it will automatically resume from last checkpoint

# Important Considerations:
# 1. Preprocess your classification dataset appropriately
# 2. Adjust model size based on your GPU capacity
# 3. Monitor GPU usage with nvidia-smi
# 4. For better generation quality, experiment with different sampling parameters